In [ ]:
import selenium
from bs4 import BeautifulSoup
import requests

url = "https://www.musinsa.com/ranking/best?period=month&age=ALL&mainCategory=002&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&priceMin=&priceMax="

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"

headers = {"User-Agent" : user_agent}
response = requests.get(url, headers)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'lxml')

from selenium import webdriver
import time

# 1. 라이브러리 가져오기
import pymysql

# 2. 접속하기
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')

# 3. 커서 가져오기
cursor = db.cursor()

# 4. musinsa 구문 만들기
SQL = """
    create table musinsa (
        ranks int not null,
        brand varchar(20) not null,
        title varchar(150) not null,
        ori_price varchar(20),
        dis_price varchar(20),
        review_count int,
        coupon varchar(20),
        sale_amt int(10),
        primary key(ranks)
);
"""

# 5. SQL 구문 실행하기
cursor.execute(SQL)

# 5-1반복으로 insert
ii = 0
driver = webdriver.Chrome()
while ii < 90 :

    # 누적판매량
    driver.get("https://www.musinsa.com/ranking/best?period=month&age=ALL&mainCategory=002&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&priceMin=&priceMax=")
    items = driver.find_elements_by_css_selector("img.lazyload.lazy")
    items[ii].click()
    time.sleep(1)
    sum_sale = driver.find_element_by_id("sales_1y_qty")
    sst = sum_sale.text

    if '만' in sst :
        if '.' in sst : 
            sst = sst.replace('.','')
            sst += '000'
        else :
            sst += '0000'
            
        sst = sst.replace('만 개 이상','')
        
    elif '천' in sst:
        if '.' in sst : 
            sst = sst.replace('.','')
            sst += '00'
        else :
            sst += '000'
        sst = sst.replace('천 개 이상','')
        
    else :
        sst = sst.replace('개 이상','')
        
    
    # 브랜드
    brands = soup.select('form#goodsRankForm div.list-box.box div.article_info p.item_title a')
    
    brd = "'"+brands[ii].text+"'"
        
    # 제품명
    names = soup.select('form#goodsRankForm div.list-box.box div.article_info p.list_info a')
    
    nm = "'"+(names[ii]['title'])+"'"
        
    # 다 넣은 곳에서 쿠폰 있는거 없는거 찾기
    all_items = soup.select('div.right_area.page-contents-best form#goodsRankForm div.list-box.box .li_box')
    
    a = all_items[ii].find('p', { 'class' : 'mu-icon mu-icon-coupon'})
    if a == None :
            coup =  "'x'"
    else :
            coup =  "'o'"
            
    # 리뷰 수 
    points = soup.select('div.right_area.page-contents-best form#goodsRankForm div.list-box.box div.article_info p.point span.count')
    
    pp = points[ii].text.replace(',','')
    
    # 가격대
    prices = soup.select('div.right_area.page-contents-best form#goodsRankForm div.list-box.box div.article_info p.price')
        
    price11 = prices[ii].text.split()
        
    if len(price11) == 2:
        pri1 = price11[0][0:-1].replace(',','')
        pri2 = price11[1][0:-1].replace(',','')
    else:
        pri1 = price11[0][0:-1].replace(',','')
        pri2 = 0
    
    # musinsa = 'INSERT INTO musinsa VALUES (',ii+1, ',', brd, ',', nm, ',', pri1, ',', pri2, ',', pp , ',', coup , ',', sst, ');'
    SQL = """INSERT INTO musinsa VALUES (
    {0}, {1}, {2}, {3}, {4}, {5}, {6}, {7});""".format(ii+1, brd, nm, pri1, pri2, pp, coup, sst)
    
    cursor.execute(SQL)
    
    ii += 1
    
    
# 6. DB에 Complete 하기
db.commit()

# 7. DB 연결 닫기
db.close()

In [ ]:
# 가격대별 아우터 순위 (1페이지)
    

In [ ]:
#  리뷰 수 많은 아우터 브랜드 순위(top 10)  

import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
select * from musinsa
    order by review_count desc 
    limit 10;
"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()

In [ ]:
# 아우터 브랜드 top 10 중 추가 할인 진행 브랜드

import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
select *
	from musinsa
	where dis_price != 0
    group by brand
    order by count(*) desc
    limit 10;
	
"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()

In [ ]:
# 브랜드별 아우터 중 쿠폰 할인 브랜드(1페이지)

import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
select *
	from musinsa
	where coupon = 'o'  
    group by brand;
"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()

In [ ]:
# 10만원 이하

import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
select *
	from musinsa
    where ori_price <= 100000
    order by ranks;
"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()

In [ ]:
# 10만원 에서 20만원 사이

import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
select *
	from musinsa
    where ori_price between 100000 and 200000
    order by ranks;
"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()

In [ ]:
# 20만원 이상

import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
select *
	from musinsa
    where ori_price >= 200000
    order by ranks;
"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()

In [ ]:
# 30만원 이상 높은 가격순 top10

import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
select *
	from musinsa
    where ori_price >= 300000
    order by ori_price desc
    limit 10;
"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()

In [ ]:
# 누적 판매량 top 10
    
import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
select *
	from musinsa
    order by sale_amt desc
    limit 10;
"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()

In [ ]:
# 5만원 ~ 20만원 누적판매량 top10

import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
select *
	from musinsa
    where ori_price between 50000 and 200000
    order by sale_amt desc
    limit 10;
"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()

In [ ]:
import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """

"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()

In [ ]:
import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """

"""
cursor.execute(sql)
result = cursor.fetchmany(90)
for record in result:
    print(record)

db.close()